In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import LabelEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# Veri setlerini oku
data = pd.read_csv("train_test.csv")
print("sütun sayısı:",data.shape[1])
print("satır sayısı:",data.shape[0])

In [22]:
# Özelliklerin belirlenmesi
features = data.drop(columns=['features'],axis=1).columns.tolist()

print(len(features))

14


In [ ]:
# One-hot encoding için kategorik değişkenleri seç
categorical_features = [ 'ilce']

# One-hot encoding işlemi
data = pd.get_dummies(data, columns=categorical_features, dummy_na=True)

# One-hot encoding sonrası oluşan sütun isimlerini al
one_hot_encoded_columns = data.columns[data.columns.str.startswith(tuple(categorical_features))]

# features listesine one-hot encoded sütun isimlerini ekle
features += list(one_hot_encoded_columns)
print(features)
len(features)

In [ ]:
# Label encoding için kategorik değişkenleri seç
categorical_features = ['ilce']

# LabelEncoder nesnesini oluşturma
label_encoder = LabelEncoder()

# Her bir kategorik değişken için label encoding uygulama
for feature in categorical_features:
    data[feature + '_encoded'] = label_encoder.fit_transform(data[feature])
    # Label encoding sonrası orijinal sütunu düşme
    data.drop(columns=[feature], inplace=True)
# Label encoding sonrası oluşan sütunları özellikler listesine ekleme
features += [feature + '_encoded' for feature in categorical_features]

print(features)
print(len(features))

In [24]:
# Bağımsız değişkenler ve bağımlı değişkenin ayrılması
X = data[features]
y = data['bagimsiz']

In [26]:
# Eğitim ve test verilerinin bölünmesi
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [27]:
# Rastgele Orman modelini seç ve eğit
model = RandomForestRegressor(n_estimators=100,random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [ ]:

# Modelin performansının değerlendirilmesi
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


In [ ]:
# Modelin performansını değerlendirme (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

In [ ]:
#İmportances features
importances = list(model.feature_importances_)
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(features, importances)]
feature_importances = sorted(feature_importances, key=lambda x: x[1], reverse = True)
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

# **Tahmin Etme**

In [28]:
# Predict verileri oku
test_data = pd.read_csv("predict.csv")

In [29]:
temp_df = pd.DataFrame(test_data['ilce'], columns=['ilce'])


In [30]:

# One-hot encoding işlemi
test_data = pd.get_dummies(test_data, columns=categorical_features, dummy_na=True)

# One-hot encoding sonrası oluşan sütunların isimlerini al
one_hot_encoded_columns_test = test_data.columns[test_data.columns.str.startswith(tuple(categorical_features))]


In [31]:
# Tahmin edilecek veriyi hazırla
X_test = test_data[features]

In [ ]:
# Tahminleri yap
predicted_values = model.predict(X_test)
print("Predicted Values:", predicted_values)

In [33]:
# Tahminleri yap
predicted_values = model.predict(X_test)

# Tahmin edilen değerleri yeni bir sütun olarak test verisine ekle
test_data_with_predictions = X_test.copy()
test_data_with_predictions['Predicted_Values'] = predicted_values

# Tahmin edilen değerleri içeren DataFrame'i göster
test_data_with_predictions

,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C,...,ilce_manisa-salihli,ilce_manisa-sarigol,ilce_manisa-saruhanli,ilce_manisa-sehzadeler,ilce_manisa-selendi,ilce_manisa-soma,ilce_manisa-turgutlu,ilce_manisa-yunusemre,ilce_nan,Predicted_Values
0,38.7996,26.9707,7.616667,10.037500,142.629167,72.116667,34.337500,4.941667,1.000000,6.395833,...,False,False,False,False,False,False,False,False,False,6.67
1,38.7996,26.9707,7.008333,19.475000,142.316667,71.862500,94.220833,3.283333,1.000000,6.508333,...,False,False,False,False,False,False,False,False,False,6.71
2,38.7996,26.9707,8.258333,21.616667,139.112500,68.158333,77.529167,3.675000,1.000000,7.837500,...,False,False,False,False,False,False,False,False,False,6.43
3,38.7996,26.9707,8.866667,26.200000,140.554167,72.541667,145.762500,1.708333,1.329167,9.379167,...,False,False,False,False,False,False,False,False,False,7.15
4,38.7996,26.9707,12.333333,16.395833,141.929167,76.854167,169.658333,3.108333,1.000000,12.854167,...,False,False,False,False,False,False,False,False,False,6.87
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358,38.6166,27.4196,13.137500,49.208333,143.004167,72.233333,157.625000,0.683333,1.712500,14.562500,...,False,False,False,False,False,False,False,True,False,13.87
1359,38.6166,27.4196,13.041667,65.825000,108.712500,68.458333,152.612500,0.825000,9.679167,14.187500,...,False,False,False,False,False,False,False,True,False,17.48
1360,38.6166,27.4196,14.216667,30.308333,171.587500,79.141667,182.120833,0.983333,1.000000,16.158333,...,False,False,False,False,False,False,False,True,False,9.13
1361,38.6166,27.4196,14.195833,32.387500,167.308333,77.558333,150.925000,0.766667,1.000000,16.070833,...,False,False,False,False,False,False,False,True,False,9.23


In [34]:
# Tarih birleştirme
test_data_with_predictions['Tarih'] = test_data['Yıl'].astype(str) + '-' + test_data['Ay'].astype(str).str.zfill(2) + '-' + test_data['Gün'].astype(str).str.zfill(2)

# ilçe ve tarih sütununu birleştirme, unique_id yapma
test_data_with_predictions['unique_id'] = test_data_with_predictions['Tarih'] + '-' + temp_df['ilce']

# Yeni bir DataFrame oluşturma
submission_df = pd.DataFrame()

# unique_id sütununu ekleme
submission_df['unique_id'] = test_data_with_predictions['unique_id']

# Bildirimsiz sum (tahmin değerleri) sütununu ekleme
submission_df['bildirimsiz_sum'] = test_data_with_predictions['Predicted_Values']

# Veriyi CSV olarak kaydetme
submission_df.to_csv('', index=False)

In [36]:
submission_df

,unique_id,bildirimsiz_sum
0,2024-02-01-izmir-aliaga,6.67
1,2024-02-02-izmir-aliaga,6.71
2,2024-02-03-izmir-aliaga,6.43
3,2024-02-04-izmir-aliaga,7.15
4,2024-02-05-izmir-aliaga,6.87
...,...,...
1358,2024-02-25-manisa-yunusemre,13.87
1359,2024-02-26-manisa-yunusemre,17.48
1360,2024-02-27-manisa-yunusemre,9.13
1361,2024-02-28-manisa-yunusemre,9.23
